In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-3.1/transformers/8b-instruct/2/model.safetensors.index.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00003-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/LICENSE
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00001-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/README.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/USE_POLICY.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer_config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00004-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/special_tokens_map.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/.gitattributes
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00002-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/gener

In [2]:
from sklearn.model_selection import train_test_split
from transformers import LlamaTokenizer, LlamaForCausalLM, Trainer, TrainingArguments
from transformers import PreTrainedTokenizerFast,AutoTokenizer,AutoModelForCausalLM
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import LlamaForCausalLM, Trainer, TrainingArguments
from IPython.display import display, Markdown
from datasets import Dataset
import re


In [3]:
# Load your training dataset (replace 'my_dataset' with your actual data
data=pd.read_csv("/kaggle/input/dataset/Quran_Grammar.csv",sep=';')


In [4]:
def clean_grammar_text(text):
    # Remove any content inside brackets including the brackets themselves
    cleaned_text = re.sub(r'\[.*?\]|\(.*?\)|\{.*?\}', '', text)
    # Remove specific characters "﴾" and "﴿"
    cleaned_text = cleaned_text.replace("﴾", "").replace("﴿", "")
    # Remove extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

In [5]:
data['grammar'] = data['grammar'].apply(clean_grammar_text)

In [6]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("ahmeddbahaa/t5-arabic-base-finetuned-wikilingua-ar")
model = AutoModelForSeq2SeqLM.from_pretrained("ahmeddbahaa/t5-arabic-base-finetuned-wikilingua-ar")

tokenizer_config.json:   0%|          | 0.00/415 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/847k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.52M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/785 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

In [7]:
class ArabicGrammarDataset(Dataset):
    def __init__(self, verses, grammars, tokenizer, max_length=512):
        self.verses = verses
        self.grammars = grammars
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.verses)

    def __getitem__(self, idx):
        if isinstance(idx, list):
        # Handle the batch case
            input_texts = [self.verses[i] for i in idx]
            target_texts = [self.grammars[i] for i in idx]
        else:
        # Handle the single case
            input_texts = [self.verses[idx]]
            target_texts = [self.grammars[idx]]

    # Tokenize inputs and targets
        inputs = self.tokenizer(input_texts, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")
        targets = self.tokenizer(target_texts, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")

    # Return tokenized inputs and targets without flattening
        return {
            'input_ids': inputs['input_ids'],  # Keep as 2D tensor
            'attention_mask': inputs['attention_mask'],  # Keep as 2D tensor
            'labels': targets['input_ids'],  # Keep as 2D tensor
        }

# Create dataset
dataset = ArabicGrammarDataset(data['verse'].tolist(), data['grammar'].tolist(), tokenizer)

In [8]:
from torch.optim import AdamW
from transformers import get_scheduler

# Define training parameters
num_epochs = 5
batch_size = 3
learning_rate = 5e-5

# Create DataLoader
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Define scheduler
num_training_steps = num_epochs * len(train_loader)
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


T5ForConditionalGeneration(
  (shared): Embedding(35000, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(35000, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [9]:
model.train()
for epoch in range(num_epochs):
    for batch in train_loader:
        # Move batch to device
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], labels=batch['labels'])
        loss = outputs.loss

        # Backward pass
        loss.backward()

        # Update parameters
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {loss.item():.4f}")


Epoch 1/5 - Loss: 1.8032
Epoch 2/5 - Loss: 1.7071
Epoch 3/5 - Loss: 2.3307
Epoch 4/5 - Loss: 1.0438
Epoch 5/5 - Loss: 0.7721


In [12]:
def generate_grammar(verse):
    model.eval()  # Set the model to evaluation mode
    input_ids = tokenizer(verse, return_tensors="pt", padding=True, truncation=True, max_length=512).input_ids.to(device)
    attention_mask = tokenizer(verse, return_tensors="pt", padding=True, truncation=True, max_length=512).attention_mask.to(device)

    # Generate the output
    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)
    
    # Decode the output
    grammar = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return grammar


In [13]:
# Sample verses to test
test_verses = [
    "بِسْمِ اللهِ الرَّحْمنِ الرَّحِيمِ",
    "الْحَمْدُ لِلّهِ رَبِّ الْعالَمِينَ"
]

# Generate grammar for each verse
for verse in test_verses:
    generated_grammar = generate_grammar(verse)
    print(f"Verse: {verse}\nGenerated Grammar: {generated_grammar}\n")

Verse: بِسْمِ اللهِ الرَّحْمنِ الرَّحِيمِ
Generated Grammar: : فعل ماض و: فاعل. : فعل ماض و:

Verse: الْحَمْدُ لِلّهِ رَبِّ الْعالَمِينَ
Generated Grammar: وَلّهِ: جار ومجرور متعلق ب. ر

